Before running the notebook you need to add the *Tick History* dataset from *Factset* to your account, via the Marketplace, the shared database must be named *tick_history*.

You also need to add the following Python packages using the **Packages** drop down:
* `snowflake.core`
* `scipy`
* `matplotlib`

In [ ]:
# Import python packages
import streamlit as st

# Snowpark
import snowflake.snowpark as S
from snowflake.snowpark import Column
import snowflake.snowpark.types as snow_types
import snowflake.snowpark.functions as snow_funcs
from snowflake.snowpark import Window
from snowflake.snowpark.context import get_active_session

# Snowflake Python API
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.stage import Stage, StageEncryption, StageDirectoryTable
from snowflake.core import Root

# Additional third-party libraries
import numpy as np
from typing import Tuple,Iterable
from scipy.stats import norm

# Get the session ie the user and role that is running this notebook
session = get_active_session()
# Print the version of Snowpark we are using
print(f"Using Snowpark: {S.__version__}")

In [ ]:
# Where we will store the aggregated ticker data
db_name = "SNOWPARK_DEMO_DB"
schema_name = "MCS_SCHEMA"
stage_name = "UDF_STAGE"

Start by creating the nessecary Snowflake objects, this is only needed once.

In [ ]:
# Create Database & Schema
root = Root(session)
demo_db = Database(name=db_name)
demo_db = root.databases.create(demo_db, mode='if_not_exists')

demo_schema = Schema(name=schema_name)
demo_schema = demo_db.schemas.create(demo_schema, mode='or_replace')

# Set context
session.use_schema(f'{db_name}.{schema_name}')

# Create Stages
udf_stage = Stage(
  name=stage_name,
  encryption=StageEncryption(type="SNOWFLAKE_SSE"), 
  directory_table=StageDirectoryTable(enable=True)
)

udf_stage = demo_schema.stages.create(udf_stage, mode='or_replace')


In [ ]:
session.use_schema(f'{db_name}.{schema_name}')
print(f"Current schema: {session.get_fully_qualified_current_schema()}")
print(f"Current role: {session.get_current_role()}")
print(f"Current warehouse: {session.get_current_warehouse()}")

The source table used, sample dataset shared by Factset, contains tick data for the previous six months for the following symbols:
* IBM-USA
* AMZN-USA
* FDS-USA
* MSFT-USA
* AAPL-USA
* META-USA

For the demo we will use the closing price for each day for one symbol so we will aggregate the tick data into one row for each symbol and day with the close price. The definition of close price is the last value for the day and we will also convert the last_date and last_time values into date and time.

In [ ]:
# Create a dataframe with closing prices
df_closing_prices_trans = (session.table('tick_history.public.th_sf_mktplace')
                                # Only use Equity trades
                                .filter((snow_funcs.col('last_price').is_not_null())
                                            & (snow_funcs.col('msg_type') == 0) # Only trades
                                            & (snow_funcs.col('SECURITY_TYPE') == 1) # only Equity
                                       )
                                # Cast last_date to Dates and last_time to Time
                                .with_columns(['trade_date', 'trade_time']
                                                      ,[snow_funcs.to_date(snow_funcs.to_char(snow_funcs.col('last_date'))
                                                                           , snow_funcs.lit('YYYYMMDD'))
                                                       ,snow_funcs.to_time(snow_funcs.to_char(snow_funcs.col('last_time'))
                                                                           , snow_funcs.lit('hhmissff'))])
                                .select('ticker', 'trade_date', 'trade_time', 'last_price', 'last_vol')
                                # Get the last value for each day as the close price
                                .with_column('closing_price'
                                             , snow_funcs.last_value(snow_funcs.col('last_price'))
                                                            .over(Window.partition_by(snow_funcs.col('ticker'), snow_funcs.col('trade_date'))
                                                                        .order_by(snow_funcs.col('trade_time'))
                                                                 )
                                            )
                                # Aggregate into one row for each symbol and day
                                .group_by(snow_funcs.col('TICKER'), snow_funcs.col('TRADE_DATE'))
                                .agg(snow_funcs.max(snow_funcs.col('CLOSING_PRICE')).alias('CLOSING_PRICE'))
                        )
# Save the data into a table so we can use this for multiple purposes
df_closing_prices_trans.write.save_as_table("closing_prices", mode="overwrite")

If we want this to run every time we get new data in tick_history.public.th_sf_mktplace we can 
instead save the dataframe as a dynamic table, below will create a Dynamic Table that will check once an hour if there is new data and then incremental update ie just add new rows.
```
df_closing_prices_trans.create_or_replace_dynamic_table("closing_prices_dt"
                                                        , warehouse="COMPUTE_WH"
                                                        , lag="1 hour"
                                                        , mode="overwrite"
                                                        , refresh_mode="INCREMENTAL"
                                                        , initialize="ON_CREATE")

```

Create a Snowpark dataframe filtered to only show close price for IBM for 2024-05-01 and forward.

Display 20 rows of it.

In [ ]:
df_closing = (session.table("closing_prices")
                        .filter((snow_funcs.col("TICKER") == 'IBM') & (snow_funcs.col("TRADE_DATE") >= '2022-05-01'))
                        .sort(snow_funcs.col("TRADE_DATE"))
             )

df_closing.sort(snow_funcs.col('TICKER'), snow_funcs.col('TRADE_DATE')).limit(20)

Plot the daily closing price over time

In [ ]:
df_closing.to_pandas().plot(x="TRADE_DATE", figsize=(6,2))

Set the number of days we are going to simulate the strock price and the number of simulations by day

In [ ]:
n_sim_runs = 10000 # intervals
n_days = 30 # iterations

A helper function to create the Snowpark Dataframe logic for calculate precent change

In [ ]:
def pct_change(indx_col: Column, val_col: Column):
    return ((val_col - snow_funcs.lag(val_col, 1).over(Window.orderBy(indx_col))) / snow_funcs.lag(val_col, 1).over(Window.orderBy(indx_col)))

Calculate historical log returns using the precent change logic

In [ ]:
df_log_returns = df_closing.select(snow_funcs.col("TRADE_DATE"), snow_funcs.col("CLOSING_PRICE")
                            ,snow_funcs.last_value(snow_funcs.col("CLOSING_PRICE")).over(Window.orderBy("TRADE_DATE")).as_("LAST_CLOSE")
                           ,snow_funcs.call_function("LN", (snow_funcs.lit(1) + pct_change(snow_funcs.col("TRADE_DATE"), snow_funcs.col("CLOSING_PRICE")))).as_("log_return"))
df_log_returns.sort("TRADE_DATE").show()

Calculate the drift as the mean for `log returns - (variance of log returns/2)` and also return the last closing price and standard dev for it. We will pull back the values and store that in local variables.

In [ ]:
params = (df_log_returns.select(snow_funcs.mean("LOG_RETURN").as_("u")
                            , snow_funcs.variance("LOG_RETURN").as_("var")
                            , snow_funcs.stddev("LOG_RETURN").as_("std_dev")
                            ,snow_funcs.max(snow_funcs.col("last_close")).as_("LAST_CLOSE"))
        .with_column("drift", (snow_funcs.col("u")-(snow_funcs.lit(0.5)*snow_funcs.col("var"))))
        .select("std_dev", "drift", "last_close")
        ).collect()

std_dev = params[0]['STD_DEV']
drift = params[0]['DRIFT']
last_close = params[0]['LAST_CLOSE']

print(f"Last close: {last_close}")
print(f"Drift: {drift}")
print(f"Std Dev: {std_dev}")

We want to calculate a simulated closing price for each iteration and day, one way to do this would be to loop through days and iterations and calculate it. This will require us to make sure we have enough memory etc and it also is a sequential process.  

By generate two dataframes, one with one row for each day and one  with one row for each simulation, we can push down the computation to Snowflake using the power of the SQL engine and compute provided to do this on scale.

In [ ]:
id_generator = snow_funcs.row_number().over(Window.order_by(snow_funcs.seq4()))

df_days = session.generator(id_generator.as_("day_id") ,rowcount=n_days)
df_sim_runs = session.generator(id_generator.as_("sim_run") ,rowcount=n_sim_runs)

We will use Numpy norm.ppf to get the random value for the simulations, since Snowflake does not have that specifc random function we can use it in a Python UDF. By setting the parameter is_permanent to True the UDF is permanent and we can later reuse it for other purposes or other users can use it (using Python, Scala, Java or SQL).

In [ ]:
@snow_funcs.udf(name="norm_ppf", is_permanent=True, replace=True, packages=["scipy"]
                , stage_location=stage_name, session=session)
def norm_ppf(pd_series: snow_types.PandasSeries[float]) -> snow_types.PandasSeries[float]:
    return norm.ppf(pd_series)


Calculate the daily return using the UDF for norm ppf, for each day and simulation. We are cross joing the dataframes with days and simulations to create a new dataframe that has one row for each day and simulation combination with the daily return. This would be equal to loop through days and iterations.

In [ ]:
df_daily_returns = (df_days.join(df_sim_runs)
                            .select("day_id", "sim_run"
                                    , snow_funcs.exp(snow_funcs.lit(drift) + snow_funcs.lit(std_dev) 
                                                        *  snow_funcs.call_function("norm_ppf", snow_funcs.uniform(0.0,1.0,snow_funcs.random()))).as_("daily_return"))
                            .sort(snow_funcs.col("DAY_ID"), snow_funcs.col("sim_run"))
                    )
df_daily_returns.limit(20)

Generate a day 0 row with return 1.0 as the starting point

In [ ]:

df_day_0 = session.generator(snow_funcs.lit(0).as_("DAY_ID"),  
                                snow_funcs.row_number().over(Window.order_by(snow_funcs.seq4())).as_("SIM_RUN")
                                , snow_funcs.lit(1.0).as_("DAILY_RETURN"), rowcount=n_sim_runs)
df_day_0.limit(10)

Union the dataframe so we only have one dataset with all rows

In [ ]:
df_simulations = df_day_0.union_all(df_daily_returns)
df_simulations.filter(snow_funcs.col("SIM_RUN") == 1).sort("DAY_ID", "SIM_RUN").limit(10)

Create a User Defined Table Function (UDTF) for calulating the simulated return, by using a UDTF we can keep track of the previous claculated value for each row and we can slo use Snowflake's capabilities in running it distirbutet.

In [ ]:
@snow_funcs.udtf(name="calc_return_udtf", is_permanent=True, replace=True
        ,packages=["typing"],  output_schema=['SIM_CLOSE'], input_types=[snow_types.FloatType(), snow_types.FloatType()]
        ,stage_location="UDF_STAGE", session=session)
class calc_return_handler:
    def __init__(self) -> None:
        self.prev_close = 0.0
    def process(self, last_close, daily_return) -> Iterable[Tuple[float]]:
        # First call we will use the last_call values,
        # for the rest prev_close
        if self.prev_close == 0.0:
            self.prev_close = last_close
        
        sim_close = self.prev_close * daily_return
        self.prev_close = sim_close # Keep track of the calculated return so we can use it for the next row
        yield (sim_close,)

Apply the UDTF on the rows, by using **partition_by** we can distrubute the calculation by simulations. In this example we have only one symbol so we just distribute by SIM_RUN, if we had more symbols we would also distrubute by each symbol.

In [ ]:
df_sim_close = (df_simulations
                    .with_column("SIM_CLOSE", snow_funcs.call_table_function("calc_return_udtf", snow_funcs.lit(last_close)
                                                                                , snow_funcs.col("DAILY_RETURN")).over(partition_by="SIM_RUN", order_by="DAY_ID"))
            )

df_sim_close.filter(snow_funcs.col("SIM_RUN") == 1).sort("DAY_ID", "SIM_RUN").limit(10)


We can save the simulations into a table to be used by others

In [ ]:
df_sim_close.write.save_as_table("ibm_30d_simulations", mode="overwrite")

Plot the min, mean and max for each day

In [ ]:
pd_local = (session.table("ibm_30d_simulations")
                    .group_by('DAY_ID')
                    .agg(snow_funcs.max(snow_funcs.col('SIM_CLOSE')).alias("MAX_CLOSE")
                            , snow_funcs.mean(snow_funcs.col('SIM_CLOSE')).alias("MEAN_CLOSE")
                            , snow_funcs.min(snow_funcs.col('SIM_CLOSE')).alias("MIN_CLOSE"))
                    .sort('DAY_ID')
                    ).to_pandas()

pd_local.plot(x='DAY_ID', figsize=(6,2))


Get the mean, Quantile (5%) and Quantile (95%)

In [ ]:
metrics = df_sim_close.select(snow_funcs.round(snow_funcs.mean(snow_funcs.col("SIM_CLOSE")), 2)
                                    , snow_funcs.round(snow_funcs.percentile_cont(0.05).within_group("SIM_CLOSE"), 2)
                                    , snow_funcs.round(snow_funcs.percentile_cont(0.95).within_group("SIM_CLOSE"), 2)).collect()
print(f"Mean price: {metrics[0][0]}")
print(f"Quantile (5%): {metrics[0][1]}")
print(f"Quantile (95%): {metrics[0][2]}")